# Merge SQLite Databases

The following code can be used to copy information from a folder of SQLite databases into a new merged SQLite database in a separate folder.

This code here is an adaptation of the [sqlMerge](https://github.com/kd8bny/sqlMerge) tool.

#### Import Packages

In [ ]:
import sqlite3
import os
import shutil

#### Set paths to InputScope databases

In [ ]:
input_folder = "./BackupFiles/"
output_folder = "./InputScope/"
db_name_ext = "inputscope.db"

#### Define Database Merging Functions

Within the `merge_databases()` function the clause `AS SELECT DISTINCT * FROM` removes duplicate rows as the tables form each database are merged:
```python
try:
    cursor_a.execute("CREATE TABLE IF NOT EXISTS " + new_table_name + " AS SELECT DISTINCT * FROM " + current_table)
    for row in cursor_b.execute("SELECT * FROM " + current_table):
```

Generally this will be the desired behaviour. However, if duplicate need to be retailed change to `AS SELECT * FROM`.

In [ ]:
def merge_databases(file_a, file_b):
    db_a = sqlite3.connect(file_a)
    db_b = sqlite3.connect(file_b)

    cursor_a = db_a.cursor()
    cursor_b = db_b.cursor()
    cursor_a.execute("SELECT name FROM sqlite_master WHERE type='table';")

    table_counter = 0
    for table_item in cursor_a.fetchall():
        current_table = table_item[0]
        
        print("Table: " + current_table)
        
        table_counter += 1

        cursor_a = db_a.cursor()

        new_table_name = current_table + "_new"

        try:
            cursor_a.execute("CREATE TABLE IF NOT EXISTS " + new_table_name + " AS SELECT DISTINCT * FROM " + current_table)
            for row in cursor_b.execute("SELECT * FROM " + current_table):
                cursor_a.execute("INSERT INTO " + new_table_name + " VALUES" + str(row) +";")

            cursor_a.execute("DROP TABLE IF EXISTS " + current_table);
            cursor_a.execute("ALTER TABLE " + new_table_name + " RENAME TO " + current_table);
            db_a.commit()

        except sqlite3.OperationalError:
            print("ERROR!: Merge Failed")
            cursor_a.execute("DROP TABLE IF EXISTS " + new_table_name);

    db_a.close()
    db_b.close()

    print("Merge Successful!")
    
    return

def read_files(directory):
    fnames = []
    for root,d_names,f_names in os.walk(directory):
        for f in f_names:
            c_name = os.path.join(root, f)
            filename, file_extension = os.path.splitext(c_name)
            if (file_extension == '.db'):
                fnames.append(c_name)
    
    return fnames

def batch_merge(input_folder, output_folder):
    
    new_db = f"./{output_folder}/{db_name_ext}"
    
    db_files = read_files(input_folder)
    shutil.copy(db_files[0], new_db)
    
    for db_file in db_files:
        
        print("Merging " + db_file)
        
        merge_databases(new_db, db_file)
        
    return

#### Call the Merge Function

In [ ]:
batch_merge(input_folder, output_folder)